In [9]:
# Importiere die erforderlichen Bibliotheken
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Dense, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input

# Lade die output.csv Datei
data = pd.read_csv("output.csv")

# Daten vorverarbeiten
features = data[['contour number', 'aspect ratio', 'extent', 'Blue', 'Green', 'Red', 'Hue']]
target = data['class']

# Teile die Daten in Trainings- und Testsets auf
train_features, test_features, train_target, test_target = train_test_split(features, target, test_size=0.2, random_state=42)

# Kodiere die Zielvariable
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_target)
test_labels = label_encoder.transform(test_target)

# Erstelle das Modell
input_shape = (train_features.shape[1],)
inputs = Input(shape=input_shape)
x = Dense(256, activation='relu')(inputs)
x = GlobalAveragePooling1D()(x)

ValueError: Input 0 of layer "global_average_pooling1d_3" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 256)